In [16]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [17]:
train_path='C:/Users/Dev Prajapati/Desktop/CNN/final_marathi_mask_splitdataset/train'
pred_path='C:/Users/Dev Prajapati/Desktop/CNN/final_marathi_mask_splitdataset/val'

In [18]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [19]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=42):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,128,128)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,128,128)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,128,128)
        self.relu1=nn.ReLU()
        #Shape= (256,12,128,128)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,64,64)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,64,64)
        self.relu2=nn.ReLU()
        #Shape= (256,20,64,64)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,64,64)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,64,64)
        self.relu3=nn.ReLU()
        #Shape= (256,32,64,64)
        
        
        self.fc=nn.Linear(in_features=112*112*32,out_features=num_classes)
        #self.sf=nn.Softmax(in_features=112 * 112 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,64,64)
            
        output=output.view(-1,32*112*112)
            
            
        output=self.fc(output)
            
        return output
            
        


In [20]:
checkpoint=torch.load('final_mask_marathi_best_checkpoint.model')#saved model
model=ConvNet(num_classes=42)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=401408, out_features=42, bias=True)
)

In [21]:
#saving in pickel file
import pickle
with open('model.pickle','wb') as f:#wb means write binary
    pickle.dump(model,f)

In [22]:
with open('model.pickle','rb') as f:#rb means read binary
    mp = pickle.load(f)

In [23]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [24]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    #print(image)
    
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)#will return array of probabilities
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred
    


In [25]:
images_path=((glob.glob(pred_path+'/**/*.jpg'))+(glob.glob(pred_path+'/**/*.jpeg')))

In [26]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [27]:
pred_dict

{'val\\a\\a_original_IMG20220331111730.jpg_174b37ad-a648-455e-8563-6000a894441a.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_3ae47059-f71a-42ff-8f49-79931876eed6.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_54d2abb4-6522-43b8-8585-62f5811e1f6e.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_5a9bb1d3-99f4-4c41-a8ca-ceab65d338d4.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_aebde7d1-fbd4-4970-8172-618882f9478b.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_b62acdcd-d013-4bbf-8a2e-7e27e579f60b.jpg': 'dh',
 'val\\a\\a_original_IMG20220331111730.jpg_c1c5160a-521e-467f-9e3c-2218a21249c6.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_c649665d-fbae-4aca-9853-89ebd49121b1.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_c8a47e4c-4995-4659-a725-c4c32fc85936.jpg': 'k',
 'val\\a\\a_original_IMG20220331111730.jpg_c8de4d6a-72ab-45d7-a8c5-c2390cdc6da5.jpg': 'a',
 'val\\a\\a_original_IMG20220331111730.jpg_cf847764-2c23-4c7f-94f0-79557373ba0a.jpg': 'a'